In [7]:
import data_extraction
import pandas as pd
import yaml, boto3

pd.set_option('display.max_columns', None)

myObj = data_extraction.DataExtractor()

s3 = boto3.client('s3')

s3.download_file("data-handling-public",'products.csv','products.csv')

In [25]:


intNumStores = int(number)

df = myObj.retrieve_stores_data(endpoint="/prod/store_details/",header= headers,totalStores=intNumStores)

type(df)

DataFrame constructor not properly called!


pandas.core.frame.DataFrame

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [26]:
df.head(5)

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent,message
0,0.0,N/A,N/A,N/A,N/A,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe,NaN
1,1.0,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe,NaN
2,2.0,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe,NaN
3,3.0,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe,NaN
4,4.0,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe,NaN


In [48]:
df.tail(5)

,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
445,445.0,Flat7StephanielakeMorrissideHP88LHCowes,50.76306,Cowes,CO-473A9FBB,94,2008-06-08,Super Store,-1.29772,GB,Europe
446,446.0,Täschestraße2539039NördlingenKirchlengern,52.20000,Kirchlengern,KI-78096E8C,61,2005-05-12,Super Store,8.63333,DE,Europe
448,448.0,Studio8MossmallWestLindaM0E6XRHighWycombe,51.62907,High Wycombe,HI-EEA7AE62,33,1998-05-14,Local,-0.74934,GB,Europe
449,449.0,Baumplatz680114KötztingBretten,49.03685,Bretten,BR-662EC74C,35,2020-10-17,Local,8.70745,DE,Europe
450,450.0,GotthilfRoseStraße7345457FeuchtwangenBadHonnef,50.64336,Bad Honnef,BA-B4AED588,36,2001-05-12,Local,7.22780,DE,eeEurope


Let's drop "lat" and "message" as they appear to be rubbish

In [31]:
df = df.drop(columns=["lat","message"])

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 1 to 450
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          418 non-null    float64       
 1   address        418 non-null    object        
 2   longitude      418 non-null    float64       
 3   locality       418 non-null    object        
 4   store_code     418 non-null    object        
 5   staff_numbers  418 non-null    object        
 6   opening_date   418 non-null    datetime64[ns]
 7   store_type     418 non-null    object        
 8   latitude       418 non-null    float64       
 9   country_code   418 non-null    object        
 10  continent      418 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 39.2+ KB


In [6]:
df.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

index seems to be an integer

In [33]:
df['index'] = pd.to_numeric(df['index'],errors="coerce")

As do latitude and longitude

In [34]:
df['longitude']= pd.to_numeric(df['longitude'],errors="coerce")
df['latitude'] = pd.to_numeric(df['latitude'],errors="coerce")

lets drop the rows where the lat and long failed 

In [35]:
df.dropna(inplace=True,axis="index",how="any") # drops the whole row if any columns have an nan

We have some date columns

In [36]:
df['opening_date']=pd.to_datetime(df['opening_date'],infer_datetime_format=True, errors='coerce')

inspect some potentially categorical data:
Country
Country Code

In [38]:
df["continent"].describe()

count        418
unique         4
top       Europe
freq         366
Name: continent, dtype: object

In [39]:
df["country_code"].describe()

count     418
unique      3
top        GB
freq      255
Name: country_code, dtype: object

Not bad, though there's 4 continents and 3 codes!

In [41]:
df["continent"] = df["continent"].astype("category")
df["country_code"] = df["country_code"].astype("category")

Store Types as categories?

In [42]:
df["store_type"].describe()

count       418
unique        4
top       Local
freq        246
Name: store_type, dtype: object

Yes. What about duplicate lines?

In [43]:
df["store_type"] = df["store_type"].astype("category")

df.duplicated().sum()

0

None! ok. Let's just check on the critical columns only - not system generated ones

In [44]:
df.duplicated(subset=["address", "locality"],keep="first").sum()

0

Still none

clean up address

In [47]:
df["address"] = df["address"].str.replace('\W', ' ', regex=True)

Now put the code into a local database maybe

In [46]:
import database_utils


myObject = database_utils.DatabaseConnector()


myObject.upload_to_db(df,"dim_store_details")

Ideas for future:
Misspellings: https://medium.com/geekculture/how-to-fix-misspelled-words-for-your-next-nlp-project-with-one-line-of-code-98888a5ff207
Common variants of companies - "ltd." vs "ltd" vs "limited"